In [1]:
import requests
import lxml.html
import pandas as pd
import os
from tqdm.notebook import tqdm
from xml.etree.ElementTree import ElementTree
from xml.etree.ElementTree import parse
from xml.etree.ElementTree import fromstring
import csv
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, WebDriverException
# from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# taplogy.com

In [2]:
# Get sitemap.xml from tapology

sitemap_url = 'https://www.tapology.com/sitemap.xml'
response = requests.get(sitemap_url)

if response.status_code == 200:
    with open('./data/sitemap.xml', 'w') as file:
        file.write(response.text)
    print('Sitemap has been successfully downloaded and saved.')
else:
    print(f'Failed to download the sitemap. HTTP status code: {response.status_code}')

Sitemap has been successfully downloaded and saved.


In [3]:
# Get fighter URL's

urls = []
base_url = 'https://www.tapology.com/fightcenter/fighters/sitemap'
total_sitemaps = 114

for i in range(1, total_sitemaps + 1):
    sitemap_url = f'{base_url}_{i}.xml' if i > 1 else f'{base_url}.xml'
    
    response = requests.get(sitemap_url)
    
    if response.status_code == 200:
        root = fromstring(response.content)
        for sitemap in root.findall('{http://www.sitemaps.org/schemas/sitemap/0.9}url'):
            loc = sitemap.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc')
            if loc is not None:
                urls.append([loc.text])
                
        # Display progress
        print(f'Processed sitemap {i}/{total_sitemaps}')

with open('./data/fighter_urls_tapology.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['URL'])
    writer.writerows(urls)

print('Finished saving fighter URLs to fighter_urls_tapology.csv')


Processed sitemap 1/114
Processed sitemap 2/114
Processed sitemap 3/114
Processed sitemap 4/114
Processed sitemap 5/114
Processed sitemap 6/114
Processed sitemap 7/114
Processed sitemap 8/114
Processed sitemap 9/114
Processed sitemap 10/114
Processed sitemap 11/114
Processed sitemap 12/114
Processed sitemap 13/114
Processed sitemap 14/114
Processed sitemap 15/114
Processed sitemap 16/114
Processed sitemap 17/114
Processed sitemap 18/114
Processed sitemap 19/114
Processed sitemap 20/114
Processed sitemap 21/114
Processed sitemap 22/114
Processed sitemap 23/114
Processed sitemap 24/114
Processed sitemap 25/114
Processed sitemap 26/114
Processed sitemap 27/114
Processed sitemap 28/114
Processed sitemap 29/114
Processed sitemap 30/114
Processed sitemap 31/114
Processed sitemap 32/114
Processed sitemap 33/114
Processed sitemap 34/114
Processed sitemap 35/114
Processed sitemap 36/114
Processed sitemap 37/114
Processed sitemap 38/114
Processed sitemap 39/114
Processed sitemap 40/114
Processed

In [5]:
# Get bout URL's

urls = []
base_url = 'https://www.tapology.com/fightcenter/bouts/sitemap'
total_sitemaps = 222  # Adjusted for the total number of bout sitemaps mentioned

for i in range(1, total_sitemaps + 1):
    sitemap_url = f'{base_url}_{i}.xml' if i > 1 else f'{base_url}.xml'
    
    response = requests.get(sitemap_url)
    
    if response.status_code == 200:
        root = fromstring(response.content)
        for sitemap in root.findall('{http://www.sitemaps.org/schemas/sitemap/0.9}url'):
            loc = sitemap.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc')
            if loc is not None:
                urls.append([loc.text])
                
        # Display progress
        print(f'Processed sitemap {i}/{total_sitemaps}')

with open('./data/bout_urls_tapology.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['URL'])
    writer.writerows(urls)

print('Finished saving bout URLs to bout_urls_tapology.csv')


KeyboardInterrupt: 

In [9]:
!open data/fighter_info.csv

# bestfightodds.com

In [2]:
# Get fighter urls 

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

# driver = webdriver.Chrome(options=chrome_options)
# driver.implicitly_wait(5)  # Implicit wait for elements to be found

# Define a function to get a new WebDriver instance
def create_driver():
    return webdriver.Chrome(options=chrome_options)

# Define a function to get fighter URL
def get_fighter_url(fighter):
    driver = create_driver()
    driver.implicitly_wait(5)  # Implicit wait for elements to be found
    try:
        # Fighter path in a href has dashes in place of spaces and dots
        path_fighter = fighter.replace(' ', '-').replace('.', '-').lower()

        # URL to search for the fighter
        url = 'https://www.bestfightodds.com/search?query=' + path_fighter

        # Navigate to the URL
        driver.get(url)

        # Find the first result link
        try:
            first_link = driver.find_element(By.XPATH, '//table/tbody/tr[1]/td[2]/a')
            first_path = first_link.get_attribute('href')
            print(fighter, first_path)
            return (fighter, first_path)
        except NoSuchElementException:
            return (fighter, None)
    
    except WebDriverException as e:
        print(f"WebDriver error occurred: {e}")
        return (fighter, None)
    
    finally:
        driver.quit()

# Read fighters from CSV file
df = pd.read_csv('./data/fighter_info.csv')

# File path for the new CSV file
output_csv_path = './data/fighter_urls_bfo.csv'

# Ensure the CSV file is created with headers if it does not exist
if not os.path.isfile(output_csv_path):
    pd.DataFrame(columns=['Fighter', 'BFO URL']).to_csv(output_csv_path, index=False)

# Load existing URLs to avoid redundant work
existing_urls = pd.read_csv(output_csv_path).set_index('Fighter')['BFO URL'].to_dict()

# Batch size and sleep time between batches
batch_size = 20
sleep_time = 3  # 1 minute sleep time between batches

# Function to process a batch of fighters
def process_batch(start_index, end_index):
    with ThreadPoolExecutor(max_workers=10) as executor:  # Increase the number of workers
        futures = []
        for i in range(start_index, end_index):
            fighter = df.loc[i, 'Fighter']
            if fighter not in existing_urls or pd.isna(existing_urls[fighter]) or existing_urls[fighter].strip() == '':
                futures.append(executor.submit(get_fighter_url, fighter))
        for future in as_completed(futures):
            fighter, url = future.result()
            if url:
                # Append URL to the CSV file
                # with open(output_csv_path, 'a') as f:
                #     f.write(f"{fighter},{url}\n")
                with open(output_csv_path, 'a') as f:
                    f.write(f"{fighter},{url if url else ''}\n")
                # Update the existing URLs dictionary
                existing_urls[fighter] = url

# Process in batches
num_fighters = len(df)
for i in tqdm(range(0, num_fighters, batch_size)):
    end_index = min(i + batch_size, num_fighters)
    process_batch(i, end_index)
    if end_index < num_fighters:
        print(f"Sleeping for {sleep_time} seconds before next batch...")
        time.sleep(sleep_time)  # Sleep between batches

# Close the WebDriver
# driver.quit()

  0%|          | 0/111 [00:00<?, ?it/s]

alexander volkov https://www.bestfightodds.com/fighters/Alexander-Volkov-2439
kelvin gastelum https://www.bestfightodds.com/fighters/Kelvin-Gastelum-4061
kyung ho kang https://www.bestfightodds.com/fighters/Kang-Kyung-Ho-12651
rinat fakhretdinov https://www.bestfightodds.com/fighters/Rinat-Fakhretdinov-13557
sergei pavlovich https://www.bestfightodds.com/fighters/Sergei-Pavlovich-8961
daniel rodriguez https://www.bestfightodds.com/fighters/Daniel-Rodriguez-6856
volkan oezdemir https://www.bestfightodds.com/fighters/Volkan-Oezdemir-4473
sharabutdin magomedov https://www.bestfightodds.com/fighters/Sharabutdin-Magomedov-18058
johnny walker https://www.bestfightodds.com/fighters/Johnny-Walker-8542
serghei spivac https://www.bestfightodds.com/fighters/Serghei-Spivac-9985
long xiao https://www.bestfightodds.com/fighters/Long-Xiao-12717
felipe lima https://www.bestfightodds.com/fighters/Felipe-Lima-10819
jose mariscal https://www.bestfightodds.com/fighters/Jose-Mariscal-8166
brendson ribeiro 

NameError: name 'driver' is not defined

In [5]:
# Now try just the first name search for those that didn't work before 

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")


# # Define a function to get a new WebDriver instance
# def create_driver():
#     return webdriver.Chrome(options=chrome_options)

# Define a function to get fighter URL by first name
def get_fighter_url_by_first_name(fighter):
    # driver = create_driver()
    # driver.implicitly_wait(3)  # Implicit wait for elements to be found
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(3)  # Implicit wait for elements to be found
    try:
        # Use only the first name
        first_name = fighter.split()[0].replace(' ', '-').replace('.', '-').lower()

        # URL to search for the fighter
        url = 'https://www.bestfightodds.com/search?query=' + first_name

        # Navigate to the URL
        driver.get(url)

        # Find the first result link
        try:
            first_link = driver.find_element(By.XPATH, '//table/tbody/tr[1]/td[2]/a')
            first_path = first_link.get_attribute('href')
            print(f"URL found for {fighter}: {first_path}")
            return (fighter, first_path)
        except NoSuchElementException:
            print(f"No URL found for {fighter} using first name search.")
            return (fighter, None)

    except WebDriverException as e:
        print(f"WebDriver error occurred for {fighter}: {e}")
        return (fighter, None)

    finally:
        driver.quit()

# Read the existing file with fighter URLs
df_urls = pd.read_csv('./data/fighter_urls_bfo.csv')

# Find fighters with NaN (missing) URLs
fighters_without_urls = df_urls[df_urls['BFO URL'].isnull()]['Fighter'].tolist()

if fighters_without_urls:
    print("Retrying fighters with only first names...")

    for fighter in fighters_without_urls:
        fighter, url = get_fighter_url_by_first_name(fighter)
        if url:
            # Update the URL in the DataFrame
            df_urls.loc[df_urls['Fighter'] == fighter, 'BFO URL'] = url

    # Save the updated URLs back to the CSV file
    df_urls.to_csv('./data/fighter_urls_bfo.csv', index=False)

    print("Updated fighter URLs saved to ./data/fighter_urls_bfo.csv.")
else:
    print("No fighters with empty URLs found.")


No fighters with empty URLs found.


In [4]:
# Try getting the rest from direct url destination
# Just first name

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

# # Define a function to get a new WebDriver instance
# def create_driver():
#     return webdriver.Chrome(options=chrome_options)

# Define a function to get fighter URL by first name
def get_fighter_url_by_first_name(fighter):
    # driver = create_driver()
    # driver.implicitly_wait(3)  # Implicit wait for elements to be found
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(5)  # Implicit wait for elements to be found
    try:
        # Use only the first name
        first_name = fighter.split()[0].replace(' ', '-').replace('.', '-').lower()

        # URL to search for the fighter
        url = 'https://www.bestfightodds.com/search?query=' + first_name

        # Navigate to the URL
        driver.get(url)

        # Check if we were redirected directly to the fighter's page
        if driver.current_url != url:
            # We were redirected, capture the URL
            print(f"Redirected to {driver.current_url} for {fighter}")
            return (fighter, driver.current_url)

        # If not redirected, find the first result link
        try:
            first_link = driver.find_element(By.XPATH, '//table/tbody/tr[1]/td[2]/a')
            first_path = first_link.get_attribute('href')
            print(f"URL found for {fighter}: {first_path}")
            return (fighter, first_path)
        except NoSuchElementException:
            print(f"No URL found for {fighter} using first name search.")
            return (fighter, None)

    except WebDriverException as e:
        print(f"WebDriver error occurred for {fighter}: {e}")
        return (fighter, None)

    finally:
        driver.quit()

# Read the existing file with fighter URLs
df_urls = pd.read_csv('./data/fighter_urls_bfo.csv')

# Find fighters with NaN (missing) URLs
fighters_without_urls = df_urls[df_urls['BFO URL'].isnull()]['Fighter'].tolist()

if fighters_without_urls:
    print("Retrying fighters with only first names...")

    for fighter in fighters_without_urls:
        fighter, url = get_fighter_url_by_first_name(fighter)
        if url:
            # Update the URL in the DataFrame
            df_urls.loc[df_urls['Fighter'] == fighter, 'BFO URL'] = url

    # Save the updated URLs back to the CSV file
    df_urls.to_csv('./data/fighter_urls_bfo.csv', index=False)

    print("Updated fighter URLs saved to ./data/fighter_urls_bfo.csv.")
else:
    print("No fighters with empty URLs found.")


No fighters with empty URLs found.


In [6]:
# Try getting the rest from direct url destination
# Full name 

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

# Define a function to get a new WebDriver instance
# def create_driver():
#     return webdriver.Chrome(options=chrome_options)

# Define a function to get fighter URL by full name
def get_fighter_url_by_full_name(fighter):
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(3)  # Implicit wait for elements to be found
    # driver = create_driver()
    # driver.implicitly_wait(5)  # Implicit wait for elements to be found
    try:
        # Use the full name
        full_name = fighter.replace(' ', '-').replace('.', '-').lower()

        # URL to search for the fighter
        url = 'https://www.bestfightodds.com/search?query=' + full_name

        # Navigate to the URL
        driver.get(url)

        # Check if we were redirected directly to the fighter's page
        if driver.current_url != url:
            # We were redirected, capture the URL
            print(f"Redirected to {driver.current_url} for {fighter}")
            return (fighter, driver.current_url)

        # If not redirected, find the first result link
        try:
            first_link = driver.find_element(By.XPATH, '//table/tbody/tr[1]/td[2]/a')
            first_path = first_link.get_attribute('href')
            print(f"URL found for {fighter}: {first_path}")
            return (fighter, first_path)
        except NoSuchElementException:
            print(f"No URL found for {fighter} using full name search.")
            return (fighter, None)

    except WebDriverException as e:
        print(f"WebDriver error occurred for {fighter}: {e}")
        return (fighter, None)

    finally:
        driver.quit()

# Read the existing file with fighter URLs
df_urls = pd.read_csv('./data/fighter_urls_bfo.csv')

# Find fighters with NaN (missing) URLs
fighters_without_urls = df_urls[df_urls['BFO URL'].isnull()]['Fighter'].tolist()

if fighters_without_urls:
    print("Retrying fighters with full names...")

    for fighter in fighters_without_urls:
        fighter, url = get_fighter_url_by_full_name(fighter)
        if url:
            # Update the URL in the DataFrame
            df_urls.loc[df_urls['Fighter'] == fighter, 'BFO URL'] = url

    # Save the updated URLs back to the CSV file
    df_urls.to_csv('./data/fighter_urls_bfo.csv', index=False)

    print("Updated fighter URLs saved to ./data/fighter_urls_bfo.csv.")
else:
    print("No fighters with empty URLs found.")


No fighters with empty URLs found.


In [7]:
# Hardcode and clean the remaining few

df_urls = pd.read_csv('./data/fighter_urls_bfo.csv')

# Hard code the URL for Abdulkareem Alselwady
df_urls.loc[df_urls['Fighter'] == 'abdulkareem alselwady', 'BFO URL'] = 'https://www.bestfightodds.com/fighters/Abdul-Kareem-Al-Selwady-10295'

# List of fighters to delete
fighters_to_delete = [
    "jinnosuke kashimura",
    "dokonjonosuke mishima",
    "keigo kunihara",
    "romie aram",
    "genki sudo",
    "masutatsu yano"
]

# Remove these fighters from the DataFrame
df_urls = df_urls[~df_urls['Fighter'].isin(fighters_to_delete)]

# Save the updated DataFrame back to the CSV file
df_urls.to_csv('./data/fighter_urls_bfo.csv', index=False)

print("Updated fighter URLs saved.")


Updated fighter URLs saved.


In [10]:
!open data/fighter_urls_bfo.csv

In [2]:
!rm -rf data/odds/
!mkdir data/odds/

In [3]:
# Scrape odds from each player URL

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

# Define a function to get a new WebDriver instance
def create_driver():
    return webdriver.Chrome(options=chrome_options)

# Define a function to scrape fighter data using Selenium
def scrape_fighter_data(url):
    driver = create_driver()
    driver.implicitly_wait(3)  # Implicit wait for elements to be found
    data = []
    try:
        driver.get(url)
        
        rows = driver.find_elements(By.XPATH, '//table[@class="team-stats-table"]/tbody/tr')
        
        for row in rows:
            if 'main-row' in row.get_attribute('class'):
                name = row.find_elements(By.XPATH, './/th[@class="oppcell"]/a')
                name = name[0].text if name else "N/A"

                open_odds = row.find_elements(By.XPATH, './/td[@class="moneyline" and @style="padding-right: 4px;"]/span')
                open_odds = open_odds[0].text if open_odds else "N/A"

                closing_odds = row.find_elements(By.XPATH, './/td[@class="moneyline" and @style="text-align: left; padding-left: 0; padding-right: 7px;"]/span')
                closing_odds = closing_odds[0].text if closing_odds else "N/A"

                event = row.find_elements(By.XPATH, './/td[@class="item-non-mobile"]/a')
                event = event[0].text if event else "N/A"

                movement = row.find_elements(By.XPATH, './/span[@class="teamPercChange"]')
                movement = movement[0].text if movement else "N/A"

                sparkline_element = row.find_elements(By.XPATH, './/td[@class="chart-cell"]')
                sparkline_data = sparkline_element[0].get_attribute('data-sparkline') if sparkline_element else "N/A"
                
                # Extract the opponent's data
                opponent = rows[rows.index(row) + 1]  # The opponent's row is the next row
                opponent_name = opponent.find_elements(By.XPATH, './/th[@class="oppcell"]/a')
                opponent_name = opponent_name[0].text if opponent_name else "N/A"

                opponent_open_odds = opponent.find_elements(By.XPATH, './/td[@class="moneyline" and @style="padding-right: 4px;"]/span')
                opponent_open_odds = opponent_open_odds[0].text if opponent_open_odds else "N/A"

                opponent_closing_odds = opponent.find_elements(By.XPATH, './/td[@class="moneyline" and @style="text-align: left; padding-left: 0"]')
                opponent_closing_odds = opponent_closing_odds[0].text if opponent_closing_odds else "N/A"

                opponent_event_date = opponent.find_elements(By.XPATH, './/td[@class="item-non-mobile" and @style="padding-left: 20px; color: #767676"]')
                opponent_event_date = opponent_event_date[0].text if opponent_event_date else "N/A"
                
                if name != "N/A" and opponent_name != "N/A":
                    data.append([opponent_event_date, event, name, open_odds, closing_odds, sparkline_data, opponent_name, opponent_open_odds, opponent_closing_odds])

    except (NoSuchElementException, WebDriverException) as e:
        print(f"Error processing {url}: {e}")
    
    finally:
        driver.quit()
    
    return data

# Function to process URL and save data
def process_url(url):
    try:
        data = scrape_fighter_data(url)
        if data:
            df_data = pd.DataFrame(data, columns=['Date', 'Event', 'Name', 'Opening Odds', 'Closing Odds', 'Sparkline Data', 'Opponent Name', 'Opponent Open Odds', 'Opponent Closing Odds'])
            
            # Extract fighter's name from the URL and use it as filename
            fighter_name = os.path.basename(url).split('.')[0].replace("-", "_")
            
            # Write DataFrame to a new CSV file
            df_data.to_csv(f'./data/odds/{fighter_name}.csv', index=False)
        else:
            print(f"No data found for {url}")
    except Exception as e:
        print(f"Error processing {url}: {e}")

# Read fighters from the CSV file
df = pd.read_csv('./data/fighter_urls_bfo.csv')

# Create a ThreadPoolExecutor to handle multiple requests concurrently
urls = [url for url in df['BFO URL'] if isinstance(url, str)]

# Use tqdm to show progress
with ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(process_url, url) for url in urls]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing URLs"):
        try:
            future.result()  # To raise any exceptions caught during processing
        except Exception as e:
            print(f"Error: {e}")

print("Scraping completed.")

Processing URLs:   0%|          | 0/2058 [00:00<?, ?it/s]

Error processing https://www.bestfightodds.com/fighters/Gregory-Rodrigues-6917: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error processing https://www.bestfightodds.com/fighters/Bobby-Green-1060: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error processing https://www.bestfightodds.com/fighters/Andre-Fili-4466: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error processing https://www.bestfightodds.com/fighters/Jean-Silva-554: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error processing https://www.bestfightodds.com/fighters/Dan-Ige-6448: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error processing https://www.bestfightodds.com/fighters/Daniel-Pineda-1240: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'

KeyboardInterrupt: 

In [ ]:
# Merge all odds files to fight_odds.csv

df_list = []

# Loop through all CSV files in 'data/odds/'
for file in os.listdir('data/odds/'):
    if file.endswith('.csv'):
        # Read the current CSV file
        df = pd.read_csv(os.path.join('data/odds/', file))
        
        # Create a new column 'BFO_ID' with the file name (without .csv)
        df['BFO_ID'] = file.replace('.csv', '')
        
        # Append the DataFrame to the list
        df_list.append(df)

# Combine all DataFrames in the list
combined_df = pd.concat(df_list, ignore_index=True)

# Ensure the DataFrame has the expected columns, including 'BFO_ID'
combined_df = combined_df.reindex(columns=[
    'Date', 'Event', 'Name', 'Opening Odds', 'Closing Odds', 
    'Sparkline Data', 'Opponent Name', 'Opponent Open Odds', 'Opponent Closing Odds', 'BFO_ID'
])

# Save the combined DataFrame to 'data/combined_odds.csv'
combined_df.to_csv('data/fight_odds.csv', index=False)

print("Combined data with BFO_ID saved to 'data/fight_odds.csv'")


In [ ]:
# Merge odds to each row in event_data_sherdog.csv

event_data_df = pd.read_csv('data/event_data_sherdog.csv')
fight_odds_df = pd.read_csv('data/fight_odds.csv')

# Convert all column names to lowercase
event_data_df.columns = event_data_df.columns.str.lower()
fight_odds_df.columns = fight_odds_df.columns.str.lower()

# Convert all string values in both DataFrames to lowercase
event_data_df = event_data_df.map(lambda s: s.lower().strip() if isinstance(s, str) else s)
fight_odds_df = fight_odds_df.map(lambda s: s.lower().strip() if isinstance(s, str) else s)

# Specify the date format for parsing
event_data_df['event date'] = pd.to_datetime(event_data_df['event date'], format='%Y-%m-%d', errors='coerce', utc=True)
fight_odds_df['date'] = pd.to_datetime(fight_odds_df['date'], format='%Y-%m-%d', errors='coerce', utc=True)

# Create two possible unique keys for each fight
event_data_df['unique_key_1'] = event_data_df.apply(lambda row: f"{row['event date'].date()}_{row['fighter 1']}_{row['fighter 2']}", axis=1)
event_data_df['unique_key_2'] = event_data_df.apply(lambda row: f"{row['event date'].date()}_{row['fighter 2']}_{row['fighter 1']}", axis=1)

fight_odds_df['unique_key'] = fight_odds_df.apply(lambda row: f"{row['date'].date()}_{row['name']}_{row['opponent name']}", axis=1)

# Remove duplicates in fight_odds_df by keeping the first occurrence of each unique key
fight_odds_df = fight_odds_df.drop_duplicates(subset=['unique_key'], keep='first')

# Merge for Fighter 1 and Fighter 2 odds using both unique keys
merged_df = pd.merge(
    event_data_df, 
    fight_odds_df[['unique_key', 'opening odds', 'closing odds']],
    left_on='unique_key_1',
    right_on='unique_key',
    how='left'
)

# Merge again using the second unique key (for swapped names)
merged_df = pd.merge(
    merged_df, 
    fight_odds_df[['unique_key', 'opening odds', 'closing odds']],
    left_on='unique_key_2',
    right_on='unique_key',
    how='left',
    suffixes=('_from_key1', '_from_key2')
)

# Assign odds based on which unique key matched
merged_df['fighter1_opening_odds'] = merged_df.apply(lambda row: row['opening odds_from_key1'] if pd.notna(row['opening odds_from_key1']) else row['opening odds_from_key2'], axis=1)
merged_df['fighter1_closing_odds'] = merged_df.apply(lambda row: row['closing odds_from_key1'] if pd.notna(row['closing odds_from_key1']) else row['closing odds_from_key2'], axis=1)

merged_df['fighter2_opening_odds'] = merged_df.apply(lambda row: row['opening odds_from_key2'] if pd.notna(row['opening odds_from_key2']) else row['closing odds_from_key1'], axis=1)
merged_df['fighter2_closing_odds'] = merged_df.apply(lambda row: row['closing odds_from_key2'] if pd.notna(row['closing odds_from_key2']) else row['opening odds_from_key1'], axis=1)

# Drop unnecessary columns
merged_df.drop(columns=['unique_key', 'unique_key_1', 'unique_key_2', 'opening odds_from_key1', 'closing odds_from_key1', 'opening odds_from_key2', 'closing odds_from_key2'], inplace=True, errors='ignore')

# Save the final merged DataFrame to a new CSV file
merged_df.to_csv('data/event_data_sherdog.csv', index=False)

# Print the summary of total, matched, and unmatched fights
total_fights = len(merged_df)
matched_fights = merged_df[['fighter1_opening_odds', 'fighter2_opening_odds']].notna().all(axis=1).sum()
unmatched_fights = total_fights - matched_fights

print(f"Total fights: {total_fights}")
print(f"Matched fights: {matched_fights}")
print(f"Unmatched fights: {unmatched_fights}")
print("Merged data saved to 'data/event_data_sherdog.csv'")

In [ ]:
# Drop extra columns from merge ^

merged_df = pd.read_csv('data/event_data_sherdog.csv')

# Drop unnecessary columns related to unique keys
columns_to_drop = ['unique_key', 'unique_key_1', 'unique_key_2', 'key_0', 'unique_key_from_key1', 'unique_key_from_key2']
merged_df.drop(columns=[col for col in columns_to_drop if col in merged_df.columns], inplace=True)

merged_df.to_csv('data/event_data_sherdog.csv', index=False)
print("Cleaned data saved to 'data/event_data_sherdog.csv'")


In [ ]:
# # Make fighter_info.csv lowercase for consistency

# file_path = './data/fighter_info.csv'
# df = pd.read_csv(file_path)

# # Convert column names to lowercase
# df.columns = [col.lower() for col in df.columns]

# # Convert all cells to lowercase
# df = df.map(lambda x: x.lower() if isinstance(x, str) else x)

# # Save the modified DataFrame to a new CSV file
# output_path = './data/fighter_info.csv'
# df.to_csv(output_path, index=False)

# print(f"All columns and cells have been converted to lowercase and saved to {output_path}.")


In [ ]:
# !open data/event_data_sherdog.csv
# !open data/fighter_info.csv

# Final Standardize/Clean

In [ ]:
# Make all files lowercase

file_paths = [
    './data/event_data_sherdog.csv',
    './data/fighter_info.csv',
    './data/github/master.csv'
]

# Process each file
for file_path in file_paths:
    if os.path.exists(file_path):
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Convert column names to lowercase
        df.columns = df.columns.str.lower()
        
        # Convert all cell values to lowercase, assuming they are strings
        df = df.applymap(lambda s: s.lower() if isinstance(s, str) else s)
        
        # Save the modified DataFrame back to the same file
        df.to_csv(file_path, index=False)
        print(f"Standardized {file_path} to lowercase.")
    else:
        print(f"File {file_path} not found.")

print("Standardization complete.")
